### Load Data

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pandas pyarrow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00


In [ ]:
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import ExactMatch

In [ ]:
data_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/data/nq_open/validation-00000-of-00001.parquet"

In [ ]:
import pandas as pd

# Load the Parquet file
df = pd.read_parquet(data_file_path, engine='pyarrow')

In [ ]:
df.head()

,question,answer
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]"
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]"
2,how many seasons of the bastard executioner ar...,"[one, one season]"
3,when did the eagles win last super bowl,[2017]
4,who won last year's ncaa women's basketball,[South Carolina]


### Load LLM

In [ ]:
!pip install -q sentencepiece

In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from collections import Counter
import numpy as np
import tensorflow as tf
import accelerate
from tensorflow import keras

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
flan_t5_large_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
flan_t5_large_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Inference

In [ ]:
PROMPT_TEMPLATE = """
Answer the following \n{question}\n
"""

In [ ]:
input_text = PROMPT_TEMPLATE.format(question='when did the eagles win last super bowl')
print(input_text)
input_ids = flan_t5_large_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = flan_t5_large_model.generate(input_ids)
print(flan_t5_large_tokenizer.decode(outputs[0]))


Answer the following 
when did the eagles win last super bowl




/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> 2017</s>


In [ ]:
def generate_answer(question, index):
    try:
        # Assuming PROMPT_TEMPLATE is defined elsewhere and uses {question}
        input_text = PROMPT_TEMPLATE.format(question=question)

        # Assuming flan_t5_large_tokenizer and flan_t5_large_model are defined and initialized elsewhere
        input_ids = flan_t5_large_tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
        outputs = flan_t5_large_model.generate(input_ids)
        answer = flan_t5_large_tokenizer.decode(outputs[0])

        print(f"Index: {index}, Question: '{question}', Generated Answer: '{answer}'")
        return answer
    except Exception as e:
        print(f"Error generating answer for question '{question}' at index {index}: {e}")
        return None

In [ ]:
print(generate_answer("when did the eagles win last super bowl", 0))

Index: 0, Question: 'when did the eagles win last super bowl', Generated Answer: '<pad> 2017</s>'
<pad> 2017</s>


In [ ]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_flan-t5-large.csv"

In [ ]:
try:
    # Try to load the DataFrame from the CSV file
    df_eval = pd.read_csv(eval_file_path)
    print("Data loaded from file.")
except FileNotFoundError:
    # If the file doesn't exist, copy df into df_eval
    df_eval = df.copy()
    if 'predictions' not in df_eval.columns:
      df_eval['predictions'] = pd.NA  # or use None or np.nan, depending on your preference
    print("File not found. Copied df into df_eval.")

File not found. Copied df into df_eval.


In [ ]:
from tqdm import tqdm

# Initialize a counter for tracking when to save the DataFrame
save_counter = 0

for index, row in tqdm(df_eval.iterrows(), total=df_eval.shape[0], desc="Generating Answers"):
    # Check if 'predictions' is empty for the current row
    if pd.isna(row['predictions']) or row['predictions'] == '':
        # Generate the answer only if 'predictions' is empty
        df_eval.at[index, 'predictions'] = generate_answer(row['question'], index)

        # Increment the save counter
        save_counter += 1

        # Save the DataFrame back to the file every 10 rows
        if save_counter % 10 == 0:
            df_eval.to_csv(eval_file_path, index=False)
            print(f"Progress saved at index {index}")

# Save any remaining progress at the end of the loop
if save_counter % 10 != 0:
    df_eval.to_csv(eval_file_path, index=False)
    print("Final progress saved.")

Generating Answers:   0%|          | 0/3610 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Generating Answers:   0%|          | 1/3610 [00:00<08:46,  6.85it/s]

Index: 0, Question: 'when was the last time anyone was on the moon', Generated Answer: '<pad> December 1972</s>'


Generating Answers:   0%|          | 3/3610 [00:00<09:44,  6.17it/s]

Index: 1, Question: 'who wrote he ain't heavy he's my brother lyrics', Generated Answer: '<pad> John Lennon</s>'
Index: 2, Question: 'how many seasons of the bastard executioner are there', Generated Answer: '<pad> 6</s>'


Generating Answers:   0%|          | 4/3610 [00:00<08:54,  6.74it/s]

...

Index: 3607, Question: 'who will take the throne after the queen dies', Generated Answer: '<pad> Charles, Prince of Wales</s>'


Generating Answers: 100%|█████████▉| 3609/3610 [13:40<00:00,  3.22it/s]

Index: 3608, Question: 'what domain has more individuals than all other domains combined do', Generated Answer: '<pad>.com</s>'


Generating Answers: 100%|██████████| 3610/3610 [13:41<00:00,  4.40it/s]

Index: 3609, Question: 'what is the meaning of the name comanche', Generated Answer: '<pad> a comanche</s>'
Progress saved at index 3609


In [ ]:
df_eval.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",<pad> December 1972</s>
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",<pad> John Lennon</s>
2,how many seasons of the bastard executioner ar...,"[one, one season]",<pad> 6</s>
3,when did the eagles win last super bowl,[2017],<pad> 2017</s>
4,who won last year's ncaa women's basketball,[South Carolina],<pad> Michigan State Spartans</s>


## Save evaluation data

In [ ]:
df_eval.to_csv(eval_file_path, index=False)

## Evaluation metrics

In [1]:
!pip install pandas pyarrow datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.7 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
eval_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_flan-t5-large.csv"

In [4]:
df = pd.read_csv(eval_file_path)

In [5]:
df.head()

,question,answer,predictions
0,when was the last time anyone was on the moon,['14 December 1972 UTC' 'December 1972'],<pad> December 1972</s>
1,who wrote he ain't heavy he's my brother lyrics,['Bobby Scott' 'Bob Russell'],<pad> John Lennon</s>
2,how many seasons of the bastard executioner ar...,['one' 'one season'],<pad> 6</s>
3,when did the eagles win last super bowl,['2017'],<pad> 2017</s>
4,who won last year's ncaa women's basketball,['South Carolina'],<pad> Michigan State Spartans</s>


In [6]:
print(df.iloc[1]['answer'])

['Bobby Scott' 'Bob Russell']


In [7]:
import ast
import string
import re
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import normalize_answer

def string_to_list(string):
    try:
        # Convert the string to a list
        matches = re.findall(r"'(.*?)'", string)
        return matches
    except Exception as e:
        # In case of an error, print the error and return the original string
        print(f"Error converting string to list: {e}")
        return string


def preprocess(text: str, answers: list) -> str:
    tokens_to_remove = ["<pad> ", "</s>"]

    # Remove specific tokens
    for token in tokens_to_remove:
        text = text.replace(token, '')

    exclude = set(string.punctuation)
    exclude.add("’")
    exclude.add("\n")

    # Remove excluded characters
    cleaned_text = normalize_answer("".join(ch for ch in text if ch not in exclude))

    # Replace text with answer item if it matches a substring in the cleaned text
    for answer_item in answers:
        if normalize_answer(answer_item) in cleaned_text:
            return answer_item  # Return the answer item that matches the substring

    return cleaned_text

In [8]:
df['answer'] = df['answer'].apply(string_to_list)
df['predictions'] = df['predictions'].str.replace("<pad> ", "")
df['predictions'] = df['predictions'].str.replace("</s>", "")
df['predictions_em'] = df.apply(lambda row: preprocess(row['predictions'], row['answer']), axis=1)

In [9]:
df.head()

,question,answer,predictions,predictions_em
0,when was the last time anyone was on the moon,"[14 December 1972 UTC, December 1972]",December 1972,December 1972
1,who wrote he ain't heavy he's my brother lyrics,"[Bobby Scott, Bob Russell]",John Lennon,john lennon
2,how many seasons of the bastard executioner ar...,"[one, one season]",6,6
3,when did the eagles win last super bowl,[2017],2017,2017
4,who won last year's ncaa women's basketball,[South Carolina],Michigan State Spartans,michigan state spartans


### Calculate Scores

In [10]:
import datasets
from drive.MyDrive.Berkeley.MIDS.DATASCI_266.project.scripts.exact_match import exact_match_score, metric_max_over_ground_truths, f1_score

df['em'] = df.apply(lambda row: metric_max_over_ground_truths(exact_match_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)
df['f1'] = df.apply(lambda row: metric_max_over_ground_truths(f1_score, row['predictions_em'], row['answer']) if row['answer'] and row['predictions_em'] else 0, axis=1)

In [11]:
!pip install -q rouge_score
!pip install -q evaluate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00


In [12]:
import evaluate

df['predictions_em'] = df['predictions_em'].astype(str)
df['answer'] = df['answer'].astype(str)

# Filter out rows where either 'predictions' or 'answer' is empty or NaN
filtered_df = df[(df['predictions_em'].notna() & df['predictions_em'].str.strip().astype(bool)) &
                 (df['answer'].notna() & df['answer'].str.strip().astype(bool))]
rouge = evaluate.load('rouge')
predictions = filtered_df['predictions_em'].tolist()
references = filtered_df['answer'].tolist()
rouge_results = rouge.compute(predictions=predictions,
                        references=references)
print(rouge_results)

{'rouge1': 0.09724295594096038, 'rouge2': 0.03305142069362382, 'rougeL': 0.09720838701618412, 'rougeLsum': 0.09698648289168905}


### Prove of correctness

#### Top 10 rows from data frame

In [ ]:
df.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",December 1972,December 1972,True,1.0
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",John Lennon,john lennon,False,0.0
2,how many seasons of the bastard executioner ar...,"['one', 'one season']",6,6,False,0.0
3,when did the eagles win last super bowl,['2017'],2017,2017,True,1.0
4,who won last year's ncaa women's basketball,['South Carolina'],Michigan State Spartans,michigan state spartans,False,0.0
5,when did the isle of wight become an island,['During the last Ice Age'],1707,1707,False,0.0
6,love yourself by justin bieber is about who,['Rihanna'],Justin Bieber,justin bieber,False,0.0
7,who was the ruler of england in 1616,['James I'],King James II,James I,True,1.0
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],Hot coffee,hot coffee,False,0.0
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],2.4 GHz,24 ghz,False,0.0


#### Top 10 rows from data frame that exactly match one of the reference answer

In [ ]:
df_em_true = df[df['em'] == True]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
0,when was the last time anyone was on the moon,"['14 December 1972 UTC', 'December 1972']",December 1972,December 1972,True,1.0
3,when did the eagles win last super bowl,['2017'],2017,2017,True,1.0
7,who was the ruler of england in 1616,['James I'],King James II,James I,True,1.0
17,who is under the mask of darth vader,['Anakin Skywalker'],Anakin Skywalker,Anakin Skywalker,True,1.0
19,what age do you need to be to buy a bb gun,['18'],18,18,True,1.0
38,what are the three fifty shades of grey books,"['Fifty Shades of Grey', 'Fifty Shades Darker'...",Fifty Shades Darker,Fifty Shades Darker,True,1.0
39,who has the power (judicial) to make decisions...,['judges'],judges,judges,True,1.0
45,what are the colors of the netherlands flag,"['blue', 'white', 'red']","red, white and blue",blue,True,1.0
51,what is the name of the main artery which take...,"['The aorta', 'aorta']",Aorta,The aorta,True,1.0
56,who sang the song good morning good morning,"['Gene Kelly', 'Connor"" ', ' ', 'Mickey Rooney']",The Beatles,,True,0.0


#### Top 10 rows from data frame that doesn't match match any of the reference answer

In [ ]:
df_em_true = df[df['em'] == False]
df_em_true.head(10)

,question,answer,predictions,predictions_em,em,f1
1,who wrote he ain't heavy he's my brother lyrics,"['Bobby Scott', 'Bob Russell']",John Lennon,john lennon,False,0.0
2,how many seasons of the bastard executioner ar...,"['one', 'one season']",6,6,False,0.0
4,who won last year's ncaa women's basketball,['South Carolina'],Michigan State Spartans,michigan state spartans,False,0.0
5,when did the isle of wight become an island,['During the last Ice Age'],1707,1707,False,0.0
6,love yourself by justin bieber is about who,['Rihanna'],Justin Bieber,justin bieber,False,0.0
8,what is the hot coffee mod in san andreas,['a normally inaccessible mini-game'],Hot coffee,hot coffee,False,0.0
9,what is the maximum data rate for the 802.11a ...,['54\\xa0Mbit/s'],2.4 GHz,24 ghz,False,0.0
10,which state is located in the centre of india,"['Chhattisgarh', 'Madhya Pradesh']",Uttar Pradesh,uttar pradesh,False,0.5
11,who sang i ran all the way home,['The Impalas'],The Script,script,False,0.0
12,where did the last name wallace come from,['a Scottish surname'],English,english,False,0.0


#### Overall Exact Match & F1 & Rouge scores

In [13]:
overall_em = sum(df['em']) / len(df['em'])
overall_f1 = sum(df['f1']) / len(df['f1'])

# Print the overall Exact Match score, F1 and Rouge score
print(f"Overall Exact Match Score: {overall_em * 100:.2f}")
print(f"Overall F1 Score: {overall_f1 * 100:.2f}")
print(f"Rouge1: {rouge_results['rouge1'] * 100:.2f}")
print(f"Rouge2: {rouge_results['rouge2'] * 100:.2f}")
print(f"RougeL: {rouge_results['rougeL'] * 100:.2f}")
print(f"RougeLSum: {rouge_results['rougeLsum'] * 100:.2f}")

Overall Exact Match Score: 8.45
Overall F1 Score: 12.63
Rouge1: 9.72
Rouge2: 3.31
RougeL: 9.72
RougeLSum: 9.70


#### Save with metrics

In [14]:
eval_with_metrics_file_path = "/content/drive/MyDrive/Berkeley/MIDS/DATASCI_266/project/eval/nq_open_flan-t5-large_metrics.csv"

In [15]:
df.to_csv(eval_with_metrics_file_path, index=False)